2150 edges, 546 nodes, 9014 edges in the line graph

In [ ]:
!pip install torch_geometric

In [ ]:
import autoencoder
from autoencoder import VGAE, GAE, EdgeDecoder, DirectedEdgeDecoder, InnerProductDecoder, DirectedInnerProductDecoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import networkx as nx
import copy
import torch
from torch_geometric.utils.convert import from_networkx
from torch_geometric.utils import train_test_split_edges
from scipy.stats import ranksums
from torch_geometric.nn import GraphConv, SAGEConv, GATConv, GCNConv
import torch.nn.functional as F
import torch.nn.functional as F2
from torch_geometric.transforms import LineGraph, RandomNodeSplit, RandomLinkSplit
from torch_geometric import seed_everything

seed_everything(42)

In [ ]:
node_rename = {node: id for id, node in enumerate(range(388, 934))}
nodefile = 'ChicagoSketch_node.tntp'
node = pd.read_csv(nodefile, sep='\t', usecols=['node', 'X', 'Y'])
flowfile = 'ChicagoSketch_flow.tntp'
colname = 'Volume '
flow = pd.read_csv(flowfile, sep='\t', usecols=['From ', 'To ', colname])

node['node'] = node['node'].map(node_rename)
node = node[node['node'].notna()]

flow['From '] = flow['From '].map(node_rename)
flow['To '] = flow['To '].map(node_rename)
flow = flow[(flow['From '].notna()) & (flow['To '].notna())]
flow.drop(flow[flow[colname] <= 0].index, inplace=True)
flow[colname] = np.log(flow[colname])

scaler = StandardScaler()
node[['X', 'Y']] = scaler.fit_transform(node[['X', 'Y']].values)
# minmax = MinMaxScaler()
# flow[[colname]] = minmax.fit_transform(flow[[colname]].values)

df = flow.rename(columns={'From ': 's', 'To ': 'r', colname: 'w'})
df1 = pd.merge(df, node, how='left', left_on='s', right_on='node')[['s', 'r', 'w', 'X', 'Y']].rename(columns={'X': 'X1', 'Y': 'Y1'})
df2 = pd.merge(df1, node, how='left', left_on='r', right_on='node')[['s', 'r', 'w', 'X1', 'Y1', 'X', 'Y']].rename(columns={'X': 'X2', 'Y': 'Y2'})
df2['feat'] = df2[['X1', 'Y1', 'X2', 'Y2']].values.tolist()

edge_name_to_y = {(s, r): w for s, r, w in df2[['s', 'r', 'w']].values}
edge_name_to_x = {(s, r): feat for s, r, feat in df2[['s', 'r', 'feat']].values}

In [ ]:
# import geopandas as gpd
# nodefile = 'anaheim_nodes.geojson'
# nodes = gpd.read_file(nodefile)
# nodes_df = pd.DataFrame(nodes)
# nodes_df[['X', 'Y']] = nodes_df['geometry'].astype(str).str.split('(').str[-1].str.split(')').str[0].str.split(' ', expand=True).astype(np.float32)
# node = nodes_df.rename(columns={'id': 'node'})

# node_rename = {node: id for id, node in enumerate(range(1, 417))}
# node['node'] = node['node'].map(node_rename)

# flowfile = 'Anaheim_flow.tntp'
# colname = 'Volume '
# flow = pd.read_csv(flowfile, sep='\t', usecols=['From ', 'To ', colname])

# flow['From '] = flow['From '].map(node_rename)
# flow['To '] = flow['To '].map(node_rename)
# flow = flow[(flow['From '].notna()) & (flow['To '].notna())]
# flow.drop(flow[flow[colname] <= 0].index, inplace=True)
# flow[colname] = np.log(flow[colname])

# scaler = StandardScaler()
# node[['X', 'Y']] = scaler.fit_transform(node[['X', 'Y']].values)
# # minmax = MinMaxScaler()
# # flow[[colname]] = minmax.fit_transform(flow[[colname]].values)

# df = flow.rename(columns={'From ': 's', 'To ': 'r', colname: 'w'})
# df1 = pd.merge(df, node, how='left', left_on='s', right_on='node')[['s', 'r', 'w', 'X', 'Y']].rename(columns={'X': 'X1', 'Y': 'Y1'})
# df2 = pd.merge(df1, node, how='left', left_on='r', right_on='node')[['s', 'r', 'w', 'X1', 'Y1', 'X', 'Y']].rename(columns={'X': 'X2', 'Y': 'Y2'})
# df2['feat'] = df2[['X1', 'Y1', 'X2', 'Y2']].values.tolist()

# edge_name_to_y = {(s, r): w for s, r, w in df2[['s', 'r', 'w']].values}
# edge_name_to_x = {(s, r): feat for s, r, feat in df2[['s', 'r', 'feat']].values}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
G = nx.from_pandas_edgelist(df2, source='s', target='r', edge_attr='w', create_using=nx.DiGraph())
airport = from_networkx(G)
airport.x = torch.from_numpy(node[['X', 'Y']].values).to(torch.float32)
print(airport)

G_line_graph = nx.line_graph(G, create_using=nx.DiGraph())
airport_line_graph = from_networkx(G_line_graph)
airport_line_graph.x = torch.from_numpy(np.vstack([edge_name_to_x[e] for e in G_line_graph.nodes])).to(torch.float32)
airport_line_graph.y = torch.from_numpy(np.vstack([edge_name_to_y[e] for e in G_line_graph.nodes])).to(torch.float32)
print(airport_line_graph)

split = RandomNodeSplit(num_val=0.1, num_test=0.4)
data = split(airport_line_graph)
data = data.to(device)

Data(edge_index=[2, 2150], w=[2150], num_nodes=541, x=[546, 2])
Data(edge_index=[2, 9014], num_nodes=2150, x=[2150, 4], y=[2150, 1])


In [ ]:
edge_array = np.array(list(dict(G_line_graph.nodes).keys()))
edge_index_train = edge_array[data.train_mask.detach().numpy()]
edge_index_val = edge_array[data.val_mask.detach().numpy()]
edge_index_calib_test = edge_array[data.test_mask.detach().numpy()]

edge_weight_train = torch.Tensor(np.stack([edge_name_to_y[tuple(edge)] for edge in edge_index_train])).to(device)
edge_weight_val = torch.Tensor(np.stack([edge_name_to_y[tuple(edge)] for edge in edge_index_val])).to(device)
edge_weight_calib_test = torch.Tensor(np.stack([edge_name_to_y[tuple(edge)] for edge in edge_index_calib_test])).to(device)

edge_index_train = torch.LongTensor(edge_index_train).T.to(device)
edge_index_val = torch.LongTensor(edge_index_val).T.to(device)
edge_index_calib_test = torch.LongTensor(edge_index_calib_test).T.to(device)
edge_tensor = torch.LongTensor(edge_array).T.to(device)

In [ ]:
edge_weight_gae_training = [edge_name_to_y[tuple(edge)] if train else 1.0 for edge, train in zip(edge_array, data.train_mask.detach().numpy())]
edge_weight_gae_training = torch.Tensor(edge_weight_gae_training).to(device) # torch.ones(edge_array.shape[0]).to(device)
# w_min, w_max = edge_weight_gae_training.min(), edge_weight_gae_training.max()
# edge_weight_gae_training = (edge_weight_gae_training - w_min) / (w_max - w_min)# * 0.5 + 0.5

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, gconv=SAGEConv):
        super().__init__()
        self.conv1 = gconv(in_channels, hidden_channels)
        self.conv2 = gconv(hidden_channels, out_channels)
        self.gconv = gconv

    def forward(self, x, edge_index, edge_weight=None):
        if self.gconv not in [GCNConv, GraphConv]:
            edge_weight = None
        if edge_weight is not None:
            edge_weight = (edge_weight).sigmoid()
        x = self.conv1(x, edge_index, edge_weight)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, gconv=SAGEConv):
        super().__init__()
        self.conv1 = gconv(in_channels, hidden_channels)
        self.conv2 = gconv(hidden_channels, out_channels)
        self.gconv = gconv

    def forward(self, x, edge_index, edge_weight=None):
        if self.gconv not in [GCNConv, GraphConv]:
            edge_weight = None
        if edge_weight is not None:
            edge_weight = (edge_weight).sigmoid()
        x = self.conv1(x, edge_index, edge_weight)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [ ]:
def cqr2_new(cal_labels2, cal_labels, cal_lower, cal_upper, val_labels2, val_labels, val_lower, val_upper, n, alpha):
    cal_scores = np.maximum((cal_labels-cal_upper)/ np.abs(cal_labels2), (cal_lower-cal_labels) / np.abs(cal_labels2))
    qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, method='higher')
    prediction_sets = [val_lower - qhat * np.abs(val_labels2), val_upper + qhat*np.abs(val_labels2)]
    cov = ((val_labels >= prediction_sets[0]) & (val_labels <= prediction_sets[1])).mean()
    # eff = np.mean(val_upper + qhat - (val_lower - qhat))
    #print(2* qhat * np.abs(val_labels2))
    eff = np.mean(prediction_sets[1] - prediction_sets[0])
    return prediction_sets, cov, eff


In [ ]:
def cqr_new2(labels2, cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n, alpha):
    cal_scores = np.maximum(cal_labels-cal_upper, cal_lower-cal_labels)
    cal_scores = cal_scores /  np.linalg.norm(labels2, ord=2)
    qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, method='higher')
    prediction_sets = [val_lower - qhat * np.linalg.norm(labels2, ord=2), val_upper + qhat * np.linalg.norm(labels2, ord=2)]
    cov = ((val_labels >= prediction_sets[0]) & (val_labels <= prediction_sets[1])).mean()
    eff = np.mean(prediction_sets[1] - prediction_sets[0])
    return prediction_sets, cov, eff

def cqr2(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n, alpha):
    cal_scores = np.maximum(cal_labels-cal_upper, cal_lower-cal_labels)
    qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, method='higher')
    prediction_sets = [val_lower - qhat, val_upper + qhat]
    cov = ((val_labels >= prediction_sets[0]) & (val_labels <= prediction_sets[1])).mean()
    eff = np.mean(val_upper + qhat - (val_lower - qhat))
    return prediction_sets, cov, eff

def qr2(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n, alpha):
    prediction_sets = [val_lower, val_upper]
    cov = ((val_labels >= prediction_sets[0]) & (val_labels <= prediction_sets[1])).mean()
    eff = np.mean(val_upper - val_lower)
    return prediction_sets, cov, eff


In [ ]:
def worst_slice_coverage(x, edge_index_calib_test, idx, val_labels, prediction_sets):
    if torch.is_tensor(x):
        x = x.detach().numpy()
    if torch.is_tensor(edge_index_calib_test):
        edge_index_calib_test = edge_index_calib_test.detach().numpy()
    xtest = np.hstack([x[edge_index_calib_test[0, ~idx]], x[edge_index_calib_test[1, ~idx]]])
    ntest = xtest.shape[0]
    nfeat = xtest.shape[1]
    xtest_test = xtest[:ntest//4]
    unitvec = np.random.randn(nfeat, 1000)
    unitvec = unitvec / np.sqrt((unitvec**2).sum(axis=0))
    # ab_range = np.quantile((xtest_test @ unitvec).flatten(), np.linspace(0, 1, 11))
    values = (xtest_test @ unitvec).flatten()
    ab_range = np.linspace(values.min(), values.max(), 10)

    ws_cov_min = None
    for delta in np.linspace(0.1, 0.5, 5):
        ws_cov = 1
        ws_a = None
        ws_b = None
        ws_vec = None
        for vec in unitvec.T:
            value_vec = xtest_test @ vec.reshape(-1, 1)
            for a, b in zip(ab_range[:-1], ab_range[1:]):
                contained = np.bitwise_and(value_vec > a, value_vec < b).flatten()
                if contained.mean() > delta:
                    conditional_cov = ((val_labels[:ntest//4][contained] >= prediction_sets[0][:ntest//4][contained]) & (val_labels[:ntest//4][contained] <= prediction_sets[1][:ntest//4][contained])).mean()
                    if conditional_cov < ws_cov:
                        print(f"Worst-Slice coverage = {conditional_cov:.4f}")
                        ws_cov = conditional_cov
                        ws_a = a
                        ws_b = b
                        ws_vec = vec
        if ws_vec is None:
            return None
        xtest_true = xtest[ntest//4:]
        value_vec = xtest_true @ ws_vec.reshape(-1, 1)
        contained = np.bitwise_and(value_vec > ws_a, value_vec < ws_b).flatten()
        ws_cov_true = ((val_labels[ntest//4:][contained] >= prediction_sets[0][ntest//4:][contained]) & (val_labels[ntest//4:][contained] <= prediction_sets[1][ntest//4:][contained])).mean()
        if ws_cov_min is not None and ws_cov_true < ws_cov_min:
            ws_cov_min = ws_cov_true
        elif ws_cov_min is None and ~np.isnan(ws_cov_true):
            ws_cov_min = ws_cov_true
    return ws_cov_min


In [ ]:
EPOCHS = 5001
ALPHA = 0.05
LR = 0.01
WD = 5e-4
HIDDEN = 8
OUT = 2

######################
SCORE = 'cqr'
GNNCONV = GraphConv
# CP_or_CQR
# Dataset
# DiGAE_or_GAE_or_LGNN

# Line Graph

In [ ]:
def train_linegraph(alpha=ALPHA, val=False):
    if val:
        model.eval()
    else:
        model.train()
    mask = data.val_mask if val else data.train_mask
    out = model(data.x, data.edge_index)
    out1=out
    label = data.y[mask]
    mse_loss = F.mse_loss(out[:, 0][mask], data.y[mask])
    low_bound = alpha / 2
    upp_bound = 1 - alpha / 2
    lower = out[:, 1][mask].reshape(-1,1)
    upper = out[:, 2][mask].reshape(-1,1)
    low_loss = torch.mean(torch.max((low_bound - 1) * (label - lower), low_bound * (label - lower)))
    upp_loss = torch.mean(torch.max((upp_bound - 1) * (label - upper), upp_bound * (label - upper)))


    loss = mse_loss/1.6 + low_loss/0.1 + upp_loss/0.05
    print('low_loss')
    print(low_loss)
    print('mse_loss')
    print(mse_loss)
    print('upp_loss')
    print(upp_loss)


    if val:
        model2.train()
    else:
        model2.eval()
    mask = data.val_mask if val else data.train_mask
    out2 = model2(data.x, data.edge_index)
    label2 = data.y[mask]-out[mask] #   new labels: X-X_hat
    label3 = upper-lower

    mse_loss2 = F.mse_loss(out2[:,0][mask].reshape(-1,1), label2)
    mse_loss3 = F.mse_loss(out2[:,1][mask].reshape(-1,1), label3)
    #low_bound = alpha / 2
    #upp_bound = 1 - alpha / 2
    #lower = out2[:, 1][mask].reshape(-1,1)
    #upper = out2[:, 2][mask].reshape(-1,1)
    #low_loss2 = torch.mean(torch.max((low_bound - 1) * (label2 - lower), low_bound * (label2 - lower)))
    #upp_loss2 = torch.mean(torch.max((upp_bound - 1) * (label2 - upper), upp_bound * (label2 - upper)))
    loss2 = mse_loss2/11.3  #+ mse_loss3/1
    print('mse_loss of model2')
    print(mse_loss2)
    print('mse_loss of model2')
    print(mse_loss3)

    if val:
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
    if not val:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return float(loss),float(loss2)

In [ ]:
def train_linegraph_fit_upperlower(alpha=ALPHA, val=False):
    if val:
        model.eval()
    else:
        model.train()
    mask = data.val_mask if val else data.train_mask
    out = model(data.x, data.edge_index)
    out1=out
    label = data.y[mask]
    mse_loss = F.mse_loss(out[:, 0][mask], data.y[mask])
    low_bound = alpha / 2
    upp_bound = 1 - alpha / 2
    lower = out[:, 1][mask].reshape(-1,1)
    upper = out[:, 2][mask].reshape(-1,1)
    low_loss = torch.mean(torch.max((low_bound - 1) * (label - lower), low_bound * (label - lower)))
    upp_loss = torch.mean(torch.max((upp_bound - 1) * (label - upper), upp_bound * (label - upper)))


    loss = mse_loss/1.6 + low_loss/0.1 + upp_loss/0.05
    print('low_loss')
    print(low_loss)
    print('mse_loss')
    print(mse_loss)
    print('upp_loss')
    print(upp_loss)


    if val:
        model2.train()
    else:
        model2.eval()
    mask = data.val_mask if val else data.train_mask
    out2 = model2(data.x, data.edge_index)
    label2 = data.y[mask]-out[mask] #   new labels: X-X_hat
    mse_loss2 = F.mse_loss(out2[mask], label2)


    #low_bound = alpha / 2
    #upp_bound = 1 - alpha / 2
    #lower = out2[:, 1][mask].reshape(-1,1)
    #upper = out2[:, 2][mask].reshape(-1,1)
    #low_loss2 = torch.mean(torch.max((low_bound - 1) * (label2 - lower), low_bound * (label2 - lower)))
    #upp_loss2 = torch.mean(torch.max((upp_bound - 1) * (label2 - upper), upp_bound * (label2 - upper)))
    loss2 = mse_loss2/11.3
    print('mse_loss of model2')
    print(mse_loss2)

    if val:
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
    if not val:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return float(loss),float(loss2)

In [ ]:
def run_conformal_regression_linegraphModel2(pred2, pred, data, n_test_calib, alpha, return_prediction_sets=False, return_conditional_coverage=False, score='cqr_new', x=None, edge_index_calib_test=None):
    num_runs = 100
    n_calib = n_test_calib // 2
    try:
        pred = pred.detach().cpu().numpy()
    except:
        pass

    try:
        pred2 = pred2.detach().cpu().numpy()
    except:
        pass

    smx = pred[data.test_mask]
    smx2= pred2[data.test_mask]
    labels = data.y[data.test_mask].detach().cpu().numpy().reshape(-1)

    #labels2= smx2.reshape(-1)

    labels2= data.y[data.test_mask].detach().cpu().numpy().reshape(-1) - smx[:, 0].reshape(-1)
    upper, lower = smx[:, 2], smx[:, 1]
    idx = np.array([1] * n_calib + [0] * (labels.shape[0]-n_calib)) > 0

    cov_all = []
    eff_all = []
    if return_conditional_coverage:
        ws_cov_all = []
    if return_prediction_sets:
        pred_set_all = []
        val_labels_all = []
        idx_all = []
    for k in range(num_runs):
        np.random.seed(k)
        np.random.shuffle(idx)
        if return_prediction_sets:
            idx_all.append(idx)
        cal_labels, val_labels = labels[idx], labels[~idx]
        cal_labels2, val_labels2= labels2[idx], labels2[~idx]
        cal_upper, val_upper = upper[idx], upper[~idx]

        cal_lower, val_lower = lower[idx], lower[~idx]
        if score == 'cqr2_new':
            prediction_sets, cov, eff = cqr2_new(cal_labels2,cal_labels,cal_lower, cal_upper,val_labels2,val_labels, val_lower, val_upper, n_test_calib, alpha)
        if return_conditional_coverage:
            #ws_cov = worst_slice_coverage(x, edge_index_calib_test, idx, val_labels, prediction_sets)
            ws_cov=[]
            if ws_cov is not None:
                ws_cov_all.append(ws_cov)
        cov_all.append(cov)
        eff_all.append(eff)
        if return_prediction_sets:
            pred_set_all.append(prediction_sets)
            val_labels_all.append(val_labels)

    if return_prediction_sets:
        if return_conditional_coverage:
            return cov_all, eff_all, ws_cov_all, pred_set_all, val_labels_all, idx_all
        return cov_all, eff_all, pred_set_all, val_labels_all, idx_all
    else:
        return np.mean(cov_all), np.mean(eff_all)

In [ ]:
def test_linegraphModel2(best_model_train2, best_model_train, alpha=ALPHA, return_prediction_sets=True, score='cqr_new2', conditional=True, x=None, calib_test_edge_index=None):
    best_model_train = best_model_train.cpu()
    best_model_train.eval()
    out = best_model_train(data.cpu().x, data.cpu().edge_index)

    best_model_train2 = best_model_train2.cpu()
    best_model_train2.eval()
    out2 = best_model_train2(data.cpu().x, data.cpu().edge_index)
    if conditional:
        return run_conformal_regression_linegraphModel2(out2, out, data.cpu(), int(data.test_mask.sum()), alpha, return_prediction_sets=return_prediction_sets, return_conditional_coverage=True, x=x, edge_index_calib_test=calib_test_edge_index, score=score)
    return run_conformal_regression_linegraphModel2(out2, out, data.cpu(), int(data.test_mask.sum()), alpha, return_prediction_sets=return_prediction_sets, score=score)

In [ ]:
model  = GNN(in_channels=data.num_features, hidden_channels=32, out_channels=3, gconv=GNNCONV).to(device)
print(model)
model2 = GNN(in_channels=data.num_features, hidden_channels=32, out_channels=2, gconv=GNNCONV).to(device)
print(model2)
optimizer  = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)
optimizer2 = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

best_val_loss = float('inf')
best_val_loss2 = float('inf')
best_model  = None
best_model2 = None
out=[]
for epoch in range(1, int(EPOCHS/1)):
    loss,loss2 = train_linegraph()
    #loss2= train_linegraphModel2()
    if epoch % 100 == 1:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch%5 ==0:
        val = True
    else:
        val=False
    val_loss, val_loss2 = train_linegraph(val)
    #val_loss2= train_linegraphModel2(labels=out,val=True)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)
        print(f'Epoch: {epoch:03d}, Best validation loss: {val_loss:.4f}')

    if val_loss2 < best_val_loss2:
        best_val_loss2 = val_loss2
        best_model2 = copy.deepcopy(model2)
        print(f'Epoch: {epoch:03d}, Best validation loss of Model 2: {val_loss2:.4f}')

GNN(
  (conv1): GraphConv(4, 32)
  (conv2): GraphConv(32, 3)
)
GNN(
  (conv1): GraphConv(4, 32)
  (conv2): GraphConv(32, 2)
)
low_loss
tensor(0.3887, grad_fn=<MeanBackward0>)
mse_loss
tensor(62.4381, grad_fn=<MseLossBackward0>)
upp_loss
tensor(2.5963, grad_fn=<MeanBackward0>)
mse_loss of model2
tensor(101.1201, grad_fn=<MseLossBackward0>)
mse_loss of model2
tensor(112.7188, grad_fn=<MseLossBackward0>)
Epoch: 001, Loss: 94.8378
low_loss
tensor(0.0812, grad_fn=<MeanBackward0>)
mse_loss
tensor(39.9882, grad_fn=<MseLossBackward0>)
upp_loss
tensor(1.8651, grad_fn=<MeanBackward0>)
mse_loss of model2
tensor(83.3969, grad_fn=<MseLossBackward0>)
mse_loss of model2
tensor(166.9591, grad_fn=<MseLossBackward0>)
Epoch: 001, Best validation loss: 63.1061
Epoch: 001, Best validation loss of Model 2: 7.3803
low_loss
tensor(0.2033, grad_fn=<MeanBackward0>)
mse_loss
tensor(25.6620, grad_fn=<MseLossBackward0>)
upp_loss
tensor(1.2947, grad_fn=<MeanBackward0>)

<ipython-input-16-de4a1ac0493a>:10: UserWarning: Using a target size (torch.Size([1075, 1])) that is different to the input size (torch.Size([1075])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse_loss = F.mse_loss(out[:, 0][mask], data.y[mask])
<ipython-input-16-de4a1ac0493a>:37: UserWarning: Using a target size (torch.Size([1075, 3])) that is different to the input size (torch.Size([1075, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse_loss2 = F.mse_loss(out2[:,0][mask].reshape(-1,1), label2)


流式输出内容被截断，只能显示最后 5000 行内容。
tensor(1.8147, grad_fn=<MseLossBackward0>)
upp_loss
tensor(0.0410, grad_fn=<MeanBackward0>)
mse_loss of model2
tensor(25.0622, grad_fn=<MseLossBackward0>)
mse_loss of model2
tensor(58.7063, grad_fn=<MseLossBackward0>)
low_loss
tensor(0.0222, grad_fn=<MeanBackward0>)
mse_loss
tensor(1.8147, grad_fn=<MseLossBackward0>)
upp_loss
tensor(0.0164, grad_fn=<MeanBackward0>)
mse_loss of model2
tensor(25.6413, grad_fn=<MseLossBackward0>)
mse_loss of model2
tensor(58.1522, grad_fn=<MseLossBackward0>)
low_loss
tensor(0.0613, grad_fn=<MeanBackward0>)
mse_loss
tensor(1.8148, grad_fn=<MseLossBackward0>)
upp_loss
tensor(0.0488, grad_fn=<MeanBackward0>)
mse_loss of model2
tensor(25.8237, grad_fn=<MseLossBackward0>)
mse_loss of model2
tensor(58.0350, grad_fn=<MseLossBackward0>)
low_loss
tensor(0.0235, grad_fn=<MeanBackward0>)
mse_loss
tensor(1.8148, grad_fn=<MseLossBackward0>)
upp_loss
tensor(0.0203, grad_fn=<MeanBackward0>)
mse_loss of model2
tensor(25.6771, grad_fn=<MseLossBa

In [ ]:
SCORE='cqr2_new'

In [ ]:
# cov_all_linegraph, eff_all_linegraph, ws_cov_all_linegraph, pred_set_all_linegraph, val_labels_all_linegraph, idx_all_linegraph = test_linegraph(best_model, x=data.x, calib_test_edge_index=edge_index_calib_test, return_prediction_sets=True, score='cqr_new', conditional=True)
cov_all_linegraph, eff_all_linegraph, pred_set_all_linegraph, val_labels_all_linegraph, idx_all_linegraph = test_linegraphModel2(best_model2, best_model, x=data.x, calib_test_edge_index=edge_index_calib_test, return_prediction_sets=True, score=SCORE, conditional=False)


In [ ]:
print(f"{np.mean(cov_all_linegraph):.4f}+/-{np.std(cov_all_linegraph):.4f}, {np.mean(eff_all_linegraph):.4f}+/-{np.std(eff_all_linegraph):.4f}")

0.9500+/-0.0147, 3.2449+/-0.0982
